In [1]:
import os

import dotenv
import pandas as pd
import openai
import pycmarkgfm
import numpy as np
from IPython import display
from ipywidgets import HTML
import torch
from transformers import BertTokenizer, BertModel
from scipy.spatial.distance import cosine

In [2]:
allgemeines_lernen = {
    "AktivTeilnehmen": 'aktiv am Unterricht teilnehmen',
    "LeistungZeigen": 'im Unterricht Leistung zeigen',
    "AufmerksamSein": 'während dem Unterricht Aufmerksam sein und den Instruktionen der Lehrperson folgen',
    "SchulinhalteMerken": 'sich schulische Inhalte merken',
    "SchulinhalteAbrufen": 'schulische Inhalte bei Bedarf abrufen',
}

In [3]:
def get_api_key():
    dotenv.load_dotenv()
    os.getenv("OPENAI_APIKEY")
    return os.getenv("OPENAI_APIKEY")

In [4]:
def _openai(messages, chain=True) -> openai.types.chat.chat_completion.Choice:
    """
    Request is sent to OpenAI
    Return the response in a ChatCompletion-Object
    """
    openai.api_key = get_api_key()
    model_result = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0.2,
        top_p=1,
    )
    response = model_result.choices[0]
    if chain:
        chain = messages + [response.message]
        return response.message.content, chain
    return response.message.content

In [5]:
def format_prompt_content(data):
    text = "\n".join(
        f" - {data['Firstname']} kann {v['assessment']} {allgemeines_lernen[k]} {v['notes']}"
        for k, v in data["allgemeines_lernen"].items()
    )
    return text

In [6]:
def prompt_for_textgeneration(data):
    system_content = f"""
    Du wirst eine Evaluation eines Schülers oder einer Schülerin bekommen.
    Die Evaluation wird in kurzen einfachen Sätzen sein.
    Dein Task ist es, diese Sätze zu einem Fliesstext zusammenzubringen.
    Der Fliesstext soll professionell sein.
    Der Fliesstext soll nur Informationen enthalten, die von der Evaluation mitgegeben werden.
    Der Fliesstext soll die Schweizer Rechtschreibung nutzen, das heisst, Scharf-S wird durch Doppel-S ersetzt.
    """
    first_prompt = f"""
    Bitte mache aus der folgenden Einschätzung einen professionellen Fliesstext:
    {format_prompt_content(data)}
    """
    return [
        {
            "role": "system",
            "content": system_content
        },
        {
            "role": "user",
            "content": """
    John Doe kann gut aktiv am Unterricht teilnehmen.
    John Doe kann gut im schulischen Umfeld Leistung zeigen.
    John Doe kann nicht immer während des Unterrichts Aufmerksam sein und den Instruktionen der Lehrperson folgen.
    John Doe kann teilweise sich schulische Inhalte merken.
    John Doe kann noch nicht schulische Inhalte bei Bedarf abrufen.
            """
        },
        {
            "role": "assistant",
            "content": """
    John Doe gelingt es gut, aktiv am Unterricht teilzunehmen.
    Ausserdem zeigt er meist gute Leistungen im schulischen Umfeld.
    Er kann jedoch während des Unterrichts nicht immer Aufmerksam sein und den Instruktionen der Lehrperson folgen.
    Ausserdem hat John Doe teilweise Mühe, sich schulische Inhalte zu merken.
    Auch gelingt es ihm noch nicht, die gemerkten Inhalte bei Bedarf abzurufen.
            """
        },
        {
            "role": "user",
            "content": """
    Jane Doe kann teilweise aktiv am Unterricht teilnehmen.
    Jane Doe kann nicht immer im schulischen Umfeld Leistung zeigen.
    Jane Doe kann gut während des Unterrichts Aufmerksam sein und den Instruktionen der Lehrperson folgen.
    Jane Doe kann gut sich schulische Inhalte merken.
    Jane Doe kann teilweise schulische Inhalte bei Bedarf abrufen.
            """
        },
        {
            "role": "assistant",
            "content": """
    Jane Doe kann nur teilweise aktiv am Unterricht teilnehmen.
    Nicht immer gelingt es ihr im schulischen Umfeld Leistungen zu zeigen.
    Sie kann jedoch gut während des Unterrichts Aufmerksam sein und den Instruktionen der Lehrperson folgen.
    Ausserdem merkt sie sich schulische Inhalte gut, kann diese jedoch nur teilweise bei Bedarf abrufen.
            """
        },
        {
            "role": "user",
            "content": first_prompt
        },
    ]

In [7]:
def get_bert_embeddings(text, model, tokenizer):
    # Ensure the input is properly formatted as a list even if it's a single string
    if isinstance(text, str):
        text = [text]  # Wrap text in a list to handle it uniformly
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state
    mean_embeddings = embeddings.mean(dim=1)  # Average across the sequence length dimension
    return mean_embeddings[0]  # Return the first (and only) item to keep it consistent

In [8]:
def calculate_similarity(text1, text2):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained('bert-base-uncased')

    embeddings1 = get_bert_embeddings(text1, model, tokenizer)
    embeddings2 = get_bert_embeddings(text2, model, tokenizer)
    
    vec1 = embeddings1.numpy()  # Should already be 1-D
    vec2 = embeddings2.numpy()  # Should already be 1-D

    similarity = 1 - cosine(vec1, vec2)
    return similarity

In [9]:
df_test = pd.read_excel("Test-Data.xlsx", "Test_Data").convert_dtypes().replace({pd.NA:None})
df_test.head()

,Firstname,AktivTeilnehmen,AktivTeilnehmenNotizen,LeistungZeigen,LeistungZeigenNotizen,AufmerksamSein,AufmerksamSeinNotizen,SchulinhalteMerken,SchulinhalteMerkenNotizen,SchulinhalteAbrufen,SchulinhalteAbrufenNotizen,Text
0,John01,gut,Durch die Mediaktion ist eine deutliche Verbes...,gut,None,gut,None,teilweise,"Man merkt, dass er zwar gute Leistungen erbrin...",teilweise,None,John01 ist im Unterricht meist aktiv dabei. In...
1,John02,gut,None,gut,None,gut,None,gut,None,gut,None,John02 ist im Unterricht meist aktiv dabei. In...
2,John03,gut,None,gut,John kommt meist gut gelaunt und sehr pünktlic...,gut,None,nicht immer,None,nicht immer,"Wenn ihn ein Thema interessiert, kann er diese...",John03 ist im Unterricht meist aktiv dabei. In...
3,John04,gut,None,gut,None,sehr gut,None,gut,None,gut,None,John04 ist im Unterricht meist aktiv dabei. In...
4,John05,nicht immer,"In einer Klassengemeinschaft, die ihm vertraut...",nicht immer,"Bei Fächer, die ihn interessieren, ist es bess...",sehr gut,"Man merkt allerdings, dass ihm manchmal die Au...",gut,None,gut,None,"John05 hat noch Schwierigkeiten, im Unterricht..."


In [18]:
test = []
for _,row in df_test.iterrows():
    original_text = row["Text"]
    firstname = row["Firstname"]
    
    input = {
        k: dict(assessment=row[k], notes=row[k+"Notizen"] or "")
        for k in allgemeines_lernen
    }
    data = dict(
        allgemeines_lernen = input,
        **{k:row[k] for k in ["Firstname"]},
    )
    prompt = prompt_for_textgeneration(data)
    openai_text, chain = _openai(prompt)
    similarity = calculate_similarity(openai_text, original_text)

    evaluation = {
        "firstname":firstname,
        "original_text":original_text,
        "openai_text":openai_text,
        "similarity":similarity
    }
    test.append(evaluation)

In [19]:
df_evaluation = pd.DataFrame(test)
df_evaluation.to_excel('Test-Evaluation.xlsx', index=False)